# Find centers

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame, points_from_xy
from numpy import diff
from pandas import DataFrame, NamedAgg
from shapely import LinearRing

from boilercv.data.sets import get_contours_df, get_dataset
from boilercv.docs import init
from boilercv.models.params import PARAMS
from boilercv.stages.experiments.e230920_subcool import EXP, GBC, THERMAL_DATA

init()

In [ ]:
TIME = "2023-09-20T17:14:18"

In [ ]:
PATH_TIME = TIME.replace(":", "-")
PATH = PARAMS.paths.experiments / f"{EXP}/{PATH_TIME}.h5"

video = get_dataset(PATH_TIME, stage="filled")["video"]

# Conversion factors
PX_PER_M = 20997.3753  # (px/m)
PX_PER_MM = PX_PER_M / 1000  # (px/mm)

# Informed by the actual water temperature data
DATA = pd.read_hdf(THERMAL_DATA)
# TODO: Timezone
SUBCOOLING = DATA.subcool[TIME]

FRAMETIME = diff(video.time.values).mean()

print(
    f"Rate = {1e3*FRAMETIME:.2f} ms",
    f"Scale = {PX_PER_MM:.2f} px/mm",
    f"Subcool = {SUBCOOLING:.2f} K",
    sep="\n",
)

Rate = 0.34 ms
Scale = 21.00 px/mm
Subcool = 1.03 K


In [ ]:
source_ds = get_dataset(PATH_TIME, stage="filled")
contours = (
    DataFrame(columns=["xpx", "ypx"])
    .assign(**get_contours_df(PATH_TIME))
    .rename(axis="columns", mapper=dict(xpx="x", ypx="y"))
    .reset_index()
    .assign(
        count=lambda df: df.groupby(["frame", "contour"], **GBC).x.transform("count")
    )
    .query("count > 3")
    .drop(columns="count")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby(["frame", "contour"], **GBC)  # type: ignore  # pyright 1.1.333
    .agg(
        x=NamedAgg(column="x", aggfunc="mean"),
        y=NamedAgg(column="x", aggfunc="mean"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
    )
)
contours

,frame,contour,x,y,centroid
0,0,0,227.9,227.9,POINT (228.015 439.375)
1,0,1,203.6,203.6,POINT (202.637 437.271)
2,0,2,224.9,224.9,POINT (225.105 407.270)
3,0,3,180.0,180.0,POINT (180.189 298.593)
4,1,0,203.9,203.9,POINT (202.624 437.240)
5,1,1,227.0,227.0,POINT (228.150 439.035)
6,1,2,224.9,224.9,POINT (224.863 405.970)
7,1,3,180.0,180.0,POINT (180.000 298.000)
8,2,0,226.9,226.9,POINT (227.863 439.122)
9,2,1,203.0,203.0,POINT (202.551 437.025)


In [ ]:
centers = (
    GeoDataFrame(contours)
    # .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
    .loc[:, ["frame", "contour", "x", "y"]]
)
centers

,frame,contour,x,y
0,0,0,227.9,227.9
1,0,1,203.6,203.6
2,0,2,224.9,224.9
3,0,3,180.0,180.0
4,1,0,203.9,203.9
5,1,1,227.0,227.0
6,1,2,224.9,224.9
7,1,3,180.0,180.0
8,2,0,226.9,226.9
9,2,1,203.0,203.0
